In [1]:
# 本文件介绍python语言本身及其相应内置语法与方法

In [2]:
# 1. iterable(可迭代对象)和iterator(迭代器)

# for loop中in后对象必须是iterable
# iterator是表示数据流的对象, 可通过__next__()方法不断获取下一个数据
# iterable是无状态的container, 必须有能力产生iterator
# iterator一定是有状态的, 但不必保存iterable
# 实现上, 当且仅当一个类的实例实现了__iter__()方法, 或是实现Sequence的基础上实现了__getitem__()方法, 才能成为一个iterable
# 这是为了保证iterable可以在__iter__()方法的作用下返回一个相应的iterator
# iterator=iter(iterable)
# 而只有在实现了__next__()方法和__iter__()方法后, 才能成为一个iterator, __next__()方法返回iterable中的下一个元素
# 下一个元素=next(iterator)
# 如果iterator到头了, 需要raise StopIteration这个exception, 这在iterator的__next__()方法中实现
# 而iterator实现__iter__()方法是为了让iterator也是一个iterable

In [4]:
# 例: 实现链表(1)

class NodeIter:
    def __init__(self, node):
        self.curr_node = node

    def __next__(self):
        if self.curr_node is None:
            raise StopIteration
        node, self.curr_node = self.curr_node, self.curr_node.next
        return node

class Node:
    def __init__(self, name):
        self.name = name
        self.next = None

    def __iter__(self):
        return NodeIter(self)  # 这里的self是一个Node

node_1 = Node("node1")
node_2 = Node("node2")
node_3 = Node("node3")
node_1.next = node_2
node_2.next = node_3


# for-in loop 其实是先管iterable要了一个iterator, 然后不断地next(iterator)
for i, node in enumerate(node_1):
    print(f"Node {i}: {node.name}")

Node 0: node1
Node 1: node2
Node 2: node3


In [ ]:
# iterator需要实现__iter__()方法是为了使得每一个子iterable都可以被调用

In [6]:
# 例: 实现链表(2) 解释为什么iterator需要是一个iterable

# 现在想要从第二个node开始print
it = iter(node_1)
first = next(it)

# 因为现在iterator不是iterable,  所以无法for-in loop
for node in it:
    print(node.name)

TypeError: 'NodeIter' object is not iterable

In [10]:
# 例: 实现链表(3)  iterator成为iterable

class NodeIter(NodeIter):

    # 几乎所有情况下, iterator实现__iter__()方法只需要返回自己即可
    # 也就是iterator = iter(iterator)
    def __iter__(self):
        return self

it = iter(node_1)
for node in it:
    print(node.name)

node1
node2
node3


In [11]:
# 2. generator(生成器)
# generator是一种特殊的iterator, 既可以用在for-in loop也可以用在next(generator)上等

# generator函数和generator对象(该对象就是一个iterator)
# 例如def gen的gen是一个generator函数, 而gen(5)的这个gen是一个generator对象
# 当python发现一个函数内包含yield语句时, python将其看待为一个generator函数
# 调用generator函数会返回一个generator对象保存在gen中
# 当对generator对象调用next()方法时, python才真正开始运行generator函数
# 从使用者的角度来看, generator和iterator的行为没有太大区别
# 而generator实际运行时, 会把自身状态保存在一个generator对象中

In [12]:
# 例: 

def gen(num):
    while num > 0:
        yield num  # 只有yield中的值会被保存
        num -= 1
    # 相当于raise StopIteration 
    # 不管return是否有返回值, generator函数都不会返回return的返回值
    # 如果要获取return的返回值, 就要catch其StopIteration
    return 100

g = gen(5)
first = next(g)
for i in g:
    print(i)

4
3
2
1


In [1]:
# 例: 实现链表(4) generator实现

class Node:
    def __init__(self, name):
        self.name = name
        self.next = None

    def __iter__(self):
        node = self
        while node is not None:
            yield node  # generator实现, 避免了额外的local variable
            node = node.next

node_1 = Node("node1")
node_2 = Node("node2")
node_3 = Node("node3")
node_1.next = node_2
node_2.next = node_3

for i, node in enumerate(node_1):
    print(f"Node {i}: {node.name}")

# generator优势: 有时和传统的迭代器相比代码简洁不少

Node 0: node1
Node 1: node2
Node 2: node3


In [2]:
# generator的send用法

In [3]:
# 例:

def gen(num):
    while num > 0:
        tmp = yield num  # 先把num返回出去, 再等待send发送回来的值赋给temp
        if tmp is not None:
            num = tmp
        num -= 1

g = gen(5)

first = next(g)
print(f"first: {first}")
print(f"send: {g.send(10)}")  # 将yield num的值认为是10, 此时tmp=10
for i in g:  # 对于generator而言, next(g)相当于g.send(None)
    print(i)

first: 5
send: 9
8
7
6
5
4
3
2
1
